In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
import torch.nn.functional as F
from torch.nn import Parameter as Param
from torch.nn import Parameter, Module
from torch.nn import Sequential, Linear, ReLU
from torch.optim import Optimizer
from torch import Tensor
from torch import nn
from torch.optim.sgd import SGD
import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv,ChebConv,GATConv, SGConv
from torch_geometric.utils import get_laplacian
from torch_geometric.utils import to_scipy_sparse_matrix,degree,add_self_loops
from torch_geometric.nn import GatedGraphConv
from torch_geometric.nn import MessagePassing, SAGEConv, GINConv, global_add_pool
from torch.optim.adam import Adam
from torch_geometric.utils import add_self_loops, degree
import os.path as osp
from typing import Callable, List, Optional, Tuple
from typing_extensions import Literal, TypedDict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from tqdm.notebook import trange, tqdm
import copy
import random
import time
import  math
import itertools
from functools import reduce
import operator
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix
from torch.utils.data import Subset
from plotnine import ggplot, geom_line, aes, xlab, theme, element_blank, ggtitle
from collections import OrderedDict
from typing_extensions import Literal, TypedDict
from torch_sparse import SparseTensor
from torch_scatter import gather_csr, scatter, segment_csr
import inspect
from torch.utils.data import DataLoader
from torch_geometric.nn.inits import uniform
device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')
from torch_geometric.typing import Adj, OptTensor

1.13.0+cpu



KeyboardInterrupt



In [ ]:
dataset=list(range(3))
i=0
for dataset_name in ['Cora','CiteSeer','PubMed']:
    transform = T.Compose([T.TargetIndegree(),
])
    path = "../Datasets/Planetoid/"+ dataset_name
    dataset[i] =Planetoid(path, dataset_name, transform=transform)
    num_nodes = dataset[i].data.num_nodes
    num_edges = dataset[i].data.num_edges // 2
    train_len = dataset[i][0].train_mask.sum()
    val_len = dataset[i][0].val_mask.sum()
    test_len = dataset[i][0].test_mask.sum()
    other_len = num_nodes - train_len - val_len - test_len
    print(f"Dataset: {dataset[i].name}")
    print(f"Num. nodes: {num_nodes} (train={train_len}, val={val_len}, test={test_len}, other={other_len})")
    print(f"Num. edges: {num_edges}")
    print(len(dataset[i][0].edge_attr))
    print(f"Num. node features: {dataset[i].num_node_features}")
    print(f"Num. classes: {dataset[i].num_classes}")
    print(f"Dataset len.: {dataset[i].len()}")
    i=i+1
    print(" ")

Dataset: Cora
Num. nodes: 2708 (train=140, val=500, test=1000, other=1068)
Num. edges: 5278
10556
Num. node features: 1433
Num. classes: 7
Dataset len.: 1
 
Dataset: CiteSeer
Num. nodes: 3327 (train=120, val=500, test=1000, other=1707)
Num. edges: 4552
9104
Num. node features: 3703
Num. classes: 6
Dataset len.: 1
 
Dataset: PubMed
Num. nodes: 19717 (train=60, val=500, test=1000, other=18157)
Num. edges: 44324
88648
Num. node features: 500
Num. classes: 3
Dataset len.: 1
 


In [2]:
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)

In [24]:
from Convs.GCN import GCNConv1
from Convs.GraphSage import GraphSage1
from Convs.GAT import GATLayer,GAT1
from Convs.GIN import GIN1
from Convs.Gated_Graph import GatedGraphConv1
from Convs.SGC import SGConv1
from Convs.SSGC import SSGConv1
from Convs.FACONV import FAConv1
from Convs.APPNP import APPNP1
from Convs.TAGConv import TAGConv1
from Convs.MessagePass import MessagePass

In [5]:
dataset[0].data
x=dataset[0].data.x
edge_index=dataset[0].data.edge_index
x_j = x[edge_index[0]]  # Source node features [num_edges, num_features]
x_i = x[edge_index[1]]  # Target node features [num_edges, num_features]
edge_weight = dataset[0][0].edge_attr

In [25]:
torch.manual_seed(1)
x = torch.randn(4, 16)
x=torch.ones(4,16)
edge_index = torch.tensor([[0, 0, 0, 1, 2, 3], [1, 2, 3, 0, 0, 0]])
row, col = edge_index
value = torch.rand(row.size(0))
adj2 = SparseTensor(row=row, col=col, value=value, sparse_sizes=(4, 4))
adj1 = adj2.set_value(None)
conv = TAGConv1(16, 32)
out1 = conv(x, edge_index)
out1

tensor([[-3.8442e-01, -1.0642e+00,  3.3893e+00,  1.9254e+00, -6.5508e-01,
         -4.1611e-01,  9.9494e-02,  2.2084e-01, -6.3247e-01, -3.6209e-02,
         -2.4360e+00, -2.5976e+00, -1.6337e+00, -3.9435e+00,  3.4731e-01,
          1.2800e+00, -7.9537e-01, -6.7870e-01,  1.6948e+00, -5.2678e-01,
         -3.2047e+00, -4.5053e+00, -3.0919e+00, -1.7887e-01, -5.9398e-01,
         -5.3636e-01,  1.8519e+00, -1.2085e+00, -8.4007e-02,  1.2789e+00,
         -1.4459e+00, -4.9096e-01],
        [ 3.4952e-03, -4.3003e-01,  1.3585e+00,  8.1173e-01, -6.8719e-01,
          6.5096e-02, -3.6358e-01, -1.1343e-01,  1.4093e-02, -1.0938e-01,
         -1.1594e+00, -1.4274e+00, -5.8295e-01, -1.8394e+00,  3.6068e-01,
          1.4522e-01, -5.0741e-01, -2.7292e-01,  1.2139e+00, -2.4361e-01,
         -2.0030e+00, -2.6594e+00, -1.6255e+00,  2.5284e-01, -1.1375e+00,
         -1.6854e-01, -1.5735e-01, -1.1076e+00, -5.1672e-01,  5.3248e-01,
          1.0541e-01, -2.7928e-01],
        [ 3.4952e-03, -4.3003e-01,  1.35

In [7]:
torch.manual_seed(0)
conv = APPNP1(K=3,alpha=0.2)
print(conv)
m=conv(x,edge_index)
m

APPNP1(K=3, alpha=0.2)


tensor([[1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088,
         1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016]])

In [8]:
torch.manual_seed(0)
conv = APPNP1(K=3,alpha=0.2)
print(conv)
m=conv(x,edge_index)
m

APPNP1(K=3, alpha=0.2)


tensor([[1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088,
         1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016]])

In [9]:
edge_index.storage

<bound method Tensor.storage of tensor([[0, 0, 0, 1, 2, 3],
        [1, 2, 3, 0, 0, 0]])>

In [10]:
torch.manual_seed(0)

conv = APPNP1(K=3,alpha=0.2)
print(conv)
m=conv(x,edge_index)
m

APPNP1(K=3, alpha=0.2)


tensor([[1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088,
         1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088, 1.2088],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016],
        [0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016,
         0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016, 0.9016]])

In [11]:
conv = FAConv1(x.shape[1],256)
print(conv)
m=conv(x, x,edge_index)
m

FAConv1(16, eps=0.1)


(tensor([[-521.7523, -521.7523, -521.7523, -521.7523, -521.7523, -521.7523,
          -521.7523, -521.7523, -521.7523, -521.7523, -521.7523, -521.7523,
          -521.7523, -521.7523, -521.7523, -521.7523],
         [-260.8261, -260.8261, -260.8261, -260.8261, -260.8261, -260.8261,
          -260.8261, -260.8261, -260.8261, -260.8261, -260.8261, -260.8261,
          -260.8261, -260.8261, -260.8261, -260.8261],
         [-260.8261, -260.8261, -260.8261, -260.8261, -260.8261, -260.8261,
          -260.8261, -260.8261, -260.8261, -260.8261, -260.8261, -260.8261,
          -260.8261, -260.8261, -260.8261, -260.8261],
         [-260.8261, -260.8261, -260.8261, -260.8261, -260.8261, -260.8261,
          -260.8261, -260.8261, -260.8261, -260.8261, -260.8261, -260.8261,
          -260.8261, -260.8261, -260.8261, -260.8261]], grad_fn=<AddBackward0>),
 (tensor([[0, 0, 0, 1, 2, 3, 0, 1, 2, 3],
          [1, 2, 3, 0, 0, 0, 0, 1, 2, 3]]),
  tensor([-0.5096, -0.5096, -0.5096, -0.5096, -0.5096, -0.50

In [12]:
type(edge_index)

torch.Tensor

In [13]:
conv = FAConv1(x.shape[1], 256)
print(conv)
m=conv(x, x,edge_index=edge_index)
m

FAConv1(16, eps=0.1)


(tensor([[-716.5657, -716.5657, -716.5657, -716.5657, -716.5657, -716.5657,
          -716.5657, -716.5657, -716.5657, -716.5657, -716.5657, -716.5657,
          -716.5657, -716.5657, -716.5657, -716.5657],
         [-358.2328, -358.2328, -358.2328, -358.2328, -358.2328, -358.2328,
          -358.2328, -358.2328, -358.2328, -358.2328, -358.2328, -358.2328,
          -358.2328, -358.2328, -358.2328, -358.2328],
         [-358.2328, -358.2328, -358.2328, -358.2328, -358.2328, -358.2328,
          -358.2328, -358.2328, -358.2328, -358.2328, -358.2328, -358.2328,
          -358.2328, -358.2328, -358.2328, -358.2328],
         [-358.2328, -358.2328, -358.2328, -358.2328, -358.2328, -358.2328,
          -358.2328, -358.2328, -358.2328, -358.2328, -358.2328, -358.2328,
          -358.2328, -358.2328, -358.2328, -358.2328]], grad_fn=<AddBackward0>),
 (tensor([[0, 0, 0, 1, 2, 3, 0, 1, 2, 3],
          [1, 2, 3, 0, 0, 0, 0, 1, 2, 3]]),
  tensor([-0.6999, -0.6999, -0.6999, -0.6999, -0.6999, -0.69

In [14]:
conv = SSGConv1(x.shape[1], 256,alpha=0.05)
print(conv)
m=conv(x, edge_index,edge_weight)
m

SSGConv1(16, 256, K=1)


RuntimeError: The size of tensor a (10560) must match the size of tensor b (10) at non-singleton dimension 0

In [ ]:
conv = SGConv1(x.shape[1], 256)
print(conv)
m=conv(x, edge_index,edge_weight)
m

SGConv1(1433, 256, K=1)


tensor([[ 0.0282, -0.0110, -0.0047,  ..., -0.0082, -0.0106, -0.0065],
        [ 0.0219, -0.0100, -0.0088,  ..., -0.0095, -0.0073, -0.0056],
        [-0.0045, -0.0390, -0.0381,  ..., -0.0132, -0.0236, -0.0123],
        ...,
        [ 0.0221, -0.0096, -0.0087,  ..., -0.0102, -0.0107, -0.0074],
        [ 0.0262, -0.0110, -0.0009,  ..., -0.0027, -0.0116, -0.0036],
        [ 0.0269, -0.0097, -0.0027,  ..., -0.0085, -0.0132, -0.0032]],
       grad_fn=<AddmmBackward0>)

In [ ]:
conv = SGConv1(x.shape[1], 256)
m=conv(x, edge_index,edge_weight)
m

tensor([[ 0.0134,  0.0050, -0.0127,  ..., -0.0273, -0.0031,  0.0202],
        [ 0.0158,  0.0067, -0.0066,  ..., -0.0231, -0.0008,  0.0248],
        [ 0.0055,  0.0357,  0.0011,  ..., -0.0298,  0.0082,  0.0343],
        ...,
        [ 0.0138,  0.0057, -0.0089,  ..., -0.0253, -0.0004,  0.0262],
        [ 0.0086,  0.0072, -0.0116,  ..., -0.0222,  0.0013,  0.0189],
        [ 0.0087,  0.0064, -0.0076,  ..., -0.0245, -0.0016,  0.0200]],
       grad_fn=<AddmmBackward0>)

In [ ]:
conv = GCNConv1(x.shape[1], 256)
m=conv(x, edge_index)
m

tensor([[-0.0333, -0.0463, -0.0333,  ...,  0.0055, -0.0163,  0.0306],
        [-0.0309, -0.0812,  0.0304,  ..., -0.0160, -0.0478,  0.0418],
        [-0.0826, -0.0194,  0.0077,  ..., -0.0236, -0.0232,  0.0069],
        ...,
        [-0.0175, -0.0398,  0.0209,  ..., -0.0173, -0.0454,  0.0493],
        [-0.0235, -0.0681,  0.0132,  ..., -0.0012,  0.0136,  0.0491],
        [-0.0127, -0.0764,  0.0246,  ..., -0.0432, -0.0053,  0.0499]],
       grad_fn=<AddBackward0>)

In [ ]:
conv = GraphSage1(x.shape[1], 256)
m=conv(x, edge_index)
m


tensor([[-0.1113, -0.0718, -0.0748,  ..., -0.0668, -0.1206,  0.0005],
        [ 0.0197,  0.0740, -0.0399,  ..., -0.0458, -0.0971,  0.1581],
        [-0.0107, -0.0934,  0.0066,  ..., -0.0191,  0.1046, -0.0282],
        ...,
        [-0.0513,  0.0912, -0.1496,  ..., -0.0477, -0.0507, -0.0589],
        [-0.0057, -0.0134, -0.0826,  ..., -0.0007,  0.0287,  0.0425],
        [-0.0680, -0.0973, -0.1591,  ..., -0.0486,  0.0016,  0.0927]],
       grad_fn=<AddBackward0>)

In [ ]:
import torch.nn.functional as F
conv = GATLayer(dataset[0].num_node_features, dataset[0].num_classes, dropout=0.0,alpha=0.0)
m=conv(x, edge_index)
m

tensor([[-0.0160, -0.0464,  0.0526,  ...,  0.0159,  0.0290,  0.0030],
        [-0.0243, -0.0315, -0.0242,  ...,  0.0179,  0.0091,  0.0469],
        [-0.0360, -0.0112, -0.0189,  ...,  0.0464,  0.0326, -0.0055],
        ...,
        [-0.0366,  0.0572,  0.0019,  ..., -0.0023,  0.0284,  0.0178],
        [-0.0380, -0.0189,  0.0064,  ...,  0.0188,  0.0342, -0.0059],
        [-0.0632,  0.0007, -0.0263,  ...,  0.0029,  0.0142,  0.0064]],
       grad_fn=<ScatterAddBackward0>)

In [ ]:

print(GAT1(dataset[0].num_node_features, dataset[0].num_classes))
print(GAT1(dataset[1].num_node_features, dataset[1].num_classes))
print(GAT1(dataset[2].num_node_features, dataset[2].num_classes))



GAT1(
  (conv1): GATLayer(
    (lin): Linear(in_features=1433, out_features=8, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
  (conv2): GATLayer(
    (lin): Linear(in_features=64, out_features=7, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
)
GAT1(
  (conv1): GATLayer(
    (lin): Linear(in_features=3703, out_features=8, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
  (conv2): GATLayer(
    (lin): Linear(in_features=64, out_features=6, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
)
GAT1(
  (conv1): GATLayer(
    (lin): Linear(in_features=500, out_features=8, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
  (conv2): GATLayer(
    (lin): Linear(in_features=64, out_features=3, bias=False)
    (leakrelu): LeakyReLU(negative_slope=0.2)
  )
)


In [ ]:
print(GCNConv(dataset[0].num_node_features, dataset[0].num_classes))

GCNConv(1433, 7)


In [21]:
#dropping out records as it is too slow 
mask = F.dropout(torch.ones(edge_index.shape[1])) > 0
edge_index_to_use = edge_index[:, mask]
#edge_attr_to_use = edge_weight[mask] if edge_weight is not None else None
(edge_index_to_use.shape)
edge_attr_to_use=None
conv = GatedGraphConv1(x.shape[1], 6000)
m=conv(x, edge_index_to_use,edge_attr_to_use)
m



tensor([[ 0.0333,  0.0405,  0.0319,  ..., -0.0028, -0.0048,  0.0111],
        [ 0.0334,  0.0463,  0.0273,  ..., -0.0099,  0.0003,  0.0049],
        [ 0.0334,  0.0463,  0.0273,  ..., -0.0099,  0.0003,  0.0049],
        [ 0.0353,  0.0436,  0.0284,  ..., -0.0070, -0.0012,  0.0064]],
       grad_fn=<AddBackward0>)